データサイエンス特別PG データサイエンス特論 第13回 主成分分析

In [ ]:
# google colab で実行する場合は下記のコメントアウトを外して実行する
# ! wget https://raw.githubusercontent.com/KHiraGit/sudspg_ds/main/ic_89_decathlon.csv

# 下記を参考に作成
# https://santakalog.com/2021/02/13/python-pca/

In [ ]:
# ライブラリをインポート
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# 十種競技のデータ(csv形式)を読み込んでデータフレームを作成
decathlon_df = pd.read_csv('ic_89_decathlon.csv', encoding='utf-8')
# ペアプロットを表示 (順位の列は除外)
sns.pairplot(decathlon_df.drop(columns='Rank'), height=0.8, aspect=1)

In [ ]:
# データを標準化
data = ['100m', 'LongJump', 'ShotPut', 'HighJump', '400m', 
        '110mH', 'DiscusThrow', 'PoleVault', 'JavelinThrow', '1500m']
stdsc = StandardScaler()
decathlon_std_df = stdsc.fit_transform(decathlon_df[data])

In [ ]:
#主成分分析を実行
pca = PCA()
pca.fit(decathlon_std_df)

# データを主成分空間に写像
pca_cor = pca.transform(decathlon_std_df)
pca_df = pd.DataFrame(pca_cor, columns = data)

# 第一主成分と第二主成分で各選手の成績をプロット
plt.figure(figsize=(6, 6))
origin =(0,0)
for x, y, name in zip(pca_cor[:, 0], pca_cor[:, 1], decathlon_df['Rank']):
    plt.annotate(s='', xy=(x,y), xytext=origin,
                 arrowprops=dict(shrink=0, width=1, headwidth=8, headlength=10, 
                 connectionstyle='arc3', facecolor='lightgray', edgecolor='lightgray'))
    plt.text(x+0.05, y, name)
plt.scatter(pca_cor[:, 0], pca_cor[:, 1])
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
# PCA の固有ベクトル
pd.DataFrame(pca.components_, columns=data, 
            index=["PC{}".format(x + 1) for x in range(len(data))])

In [ ]:
# 固有ベクトルを棒グラフで表示
centers = pd.DataFrame(pca.components_, columns=data)
f, axes = plt.subplots(len(data), 1, sharex=True, figsize=(10,10))
for i, ax in enumerate(axes):
    center = centers.loc[i, :]
    maxPC = 1.1 * np.max(np.max(np.abs(center)))
    center.plot.bar(ax=ax)
    ax.axhline(color='#cccccc')
    ax.set_ylabel(f'PC{i}')
    ax.set_ylim(-maxPC, maxPC)

In [ ]:
# 第一主成分と第二主成分における観測変数をプロット
plt.figure(figsize=(6, 6))
origin =(0,0)
for x, y, name in zip(pca.components_[0], pca.components_[1], data):
    plt.annotate(s='', xy=(x,y), xytext=origin,
                 arrowprops=dict(shrink=0, width=1, headwidth=8, headlength=10, 
                 connectionstyle='arc3', facecolor='lightgray', edgecolor='lightgray'))
    plt.text(x, y, name)
plt.scatter(pca.components_[0], pca.components_[1], alpha=0.8)
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
# 寄与率
_output = ''
for i in range(10):
    _output = _output + f'[PC{i+1} {pca.explained_variance_ratio_[i]:.03f}] '
print(_output)

In [ ]:
# 累積寄与率
plt.plot([0] + list( np.cumsum(pca.explained_variance_ratio_)), "-o")
plt.grid()
plt.show()